<font size=6>KNN (K-Nearest Neighbor)</font><br>
<font size=5>Autor: <font color="#2890f9"> María Camila Gómez Hernández <br></font>
<font size=3>GitHub: <font color="#A371F7"> @TrashCam9 <br></font>
<font size=3>Cuenta uniandes: <font color="#FFF200"> mc.gomezh1</font>

K-nearest neighbor es un algoritmo de aprendizaje supervisado que puede ser usado para problemas de regresión y de clasificación. KNN funciona en el principio de asumir que cualquier datos cercano a otro cae en la misma clase, clasifica un nuevo dato basado en la similitud.

In [37]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

import matplotlib.pyplot as plt
import seaborn as sns

# 1. Carga de datos

In [ ]:
data = pd.read_csv('./data/datos_convertidos.csv')

In [ ]:
data.shape

In [ ]:
data.head()

# 2. Entedimiento de los datos

Al realizar el preparamiento de los datos cambios los tipos de algunas columnas para que nuestro algoritmo pudiera procesarlas de buena forma.

In [ ]:
data.dtypes

In [ ]:
data.describe()

# 3. Limpieza y preparación de los datos

El proceso de limpieza lo realizamos en el notebook llamado [PrepDatos.ipynb](PrepDatos.ipynb) 

Graficamos nuestra variable objetivo

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.title('# personas que han reportado CHD o MI')
sns.countplot(x='HeartDisease', data=data)

### Selección de atributos / features para la clasificación

Como en el preprocesamiento todas las variables se convertieron de categoricas a numericas, podemos usarlas directamente en el modelo. Eliminamos de la lista que creamos con todas las columnas a nuestra variable objetivo y a la columna index.

In [ ]:
features = list(data.columns)
features.remove('Index')
features.remove('HeartDisease')

### Particionamiento del conjunto de datos en entrenamiento y prueba

Utilizamos la estrategia hold-out

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data[features], data['HeartDisease'], test_size = 0.2, random_state = 0)

In [ ]:
print('Tamaño del conjunto original:', data.shape[0])
print('Tamaño del conjunto de entrenamiento:', X_train.shape[0])
print('Tamaño del conjunto de prueba:', X_test.shape[0])

In [ ]:
Y_train.value_counts(normalize = True)

In [ ]:
Y_test.value_counts(normalize = True)

# 4. Entrenamiento de primer modelo con KNN

Usaremos el default value de n_neighbors para el primer modelo (3).

In [ ]:
knn_model = KNeighborsClassifier()

Entrenamiento del modelo:

In [ ]:
knn_model.fit(X_train, Y_train)

Generación de predicciones para entrenamiento y test

In [ ]:
preds_train = knn_model.predict(X_train)
preds_test = knn_model.predict(X_test)

Revisamos la accuracy del modelo en abmos conjuntos en terminos porcentuales

In [ ]:
print('Accuracy (train):', accuracy_score(Y_train, preds_train))
print('Accuracy (test):', accuracy_score(Y_test, preds_test))

Construimos la matriz de confusión para saber si se equivocó más prediciendo la clase positiva o negativa.

In [ ]:
cm_train = confusion_matrix(Y_train, preds_train, labels = knn_model.classes_)
cm_test = confusion_matrix(Y_test, preds_test, labels = knn_model.classes_)
cm_train_norm = confusion_matrix(Y_train, preds_train, labels = knn_model.classes_, normalize = 'true')
cm_test_norm = confusion_matrix(Y_test, preds_test, labels = knn_model.classes_, normalize = 'true')

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (13, 10))
ConfusionMatrixDisplay(confusion_matrix = cm_train, display_labels = knn_model.classes_).plot(ax = ax1, values_format = ',.0f')
ConfusionMatrixDisplay(confusion_matrix = cm_train_norm, display_labels = knn_model.classes_).plot(ax = ax2, values_format = '.3f')
ConfusionMatrixDisplay(confusion_matrix = cm_test, display_labels = knn_model.classes_).plot(ax = ax3, values_format = ',.0f')
ConfusionMatrixDisplay(confusion_matrix = cm_test_norm, display_labels = knn_model.classes_).plot(ax = ax4, values_format = '.3f')

ax1.set_title('Train')
ax2.set_title('Train (normalized)')
ax3.set_title('Test')
ax4.set_title('Test (normalized)')
plt.show()

Podemos ver que nuestro modelo está prediciendo la mayoria de los datos como negativos, incluso si son positivos.

Calculamos la metrica f1 para las siguientes comparaciones de los modelos

In [ ]:
pd.DataFrame([
    {
        'Train': f1_score(Y_train, preds_train),
        'Test': f1_score(Y_test, preds_test)
    }
], index = ['F1'])

### Estandarización de datos para predicción

In [ ]:
X_train[features].head()

En este ejemplo revisamos las graficas normales y estandarizadas para ver el cambio de las metricas en BMI.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (13, 5))
X_train['BMI'].hist(bins = 30, ax = ax1)
((X_train['BMI'] - X_train['BMI'].mean()) / X_train['BMI'].std()).hist(bins = 30, ax = ax2)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

# 5. Entrenamiento de segundo modelo basado en KNN con datos estandarizados

Hacemos el proceso anterior de el fit y predict, con k = 3

In [ ]:
knn_model.fit(scaler.transform(X_train), Y_train)

In [ ]:
preds_train = knn_model.predict(scaler.transform(X_train))
preds_test = knn_model.predict(scaler.transform(X_test))

In [ ]:
cm_train = confusion_matrix(Y_train, preds_train, labels = knn_model.classes_)
cm_test = confusion_matrix(Y_test, preds_test, labels = knn_model.classes_)
cm_train_norm = confusion_matrix(Y_train, preds_train, labels = knn_model.classes_, normalize = 'true')
cm_test_norm = confusion_matrix(Y_test, preds_test, labels = knn_model.classes_, normalize = 'true')

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (13, 10))
ConfusionMatrixDisplay(confusion_matrix = cm_train, display_labels = knn_model.classes_).plot(ax = ax1, values_format = ',.0f')
ConfusionMatrixDisplay(confusion_matrix = cm_train_norm, display_labels = knn_model.classes_).plot(ax = ax2, values_format = '.3f')
ConfusionMatrixDisplay(confusion_matrix = cm_test, display_labels = knn_model.classes_).plot(ax = ax3, values_format = ',.0f')
ConfusionMatrixDisplay(confusion_matrix = cm_test_norm, display_labels = knn_model.classes_).plot(ax = ax4, values_format = '.3f')

ax1.set_title('Train')
ax2.set_title('Train (normalized)')
ax3.set_title('Test')
ax4.set_title('Test (normalized)')
plt.show()

In [ ]:
pd.DataFrame([
    {
        'Train': f1_score(Y_train, preds_train),
        'Test': f1_score(Y_test, preds_test)
    }
], index = ['F1'])

Se puede notar que con la estandarización la predicción es un poco mejor pero aun así hay demasiados positivos que son detectados falsos.

# 6. Busqueda de mejor modelo con GridSearchCV

Podemos cambiar de k para saber qué K es más apropiado para nuestro modelo. Utilizarenos f1 para saber cual es mejor.

In [ ]:
k_range = list(range(1,10))

In [ ]:
param_grid = dict(n_neighbors = k_range)
param_grid['weights'] = ['uniform', 'distance']
param_grid['p'] = [1, 2]

In [ ]:
grid = GridSearchCV(knn_model, param_grid, cv = 10, scoring = ['f1', 'precision', 'recall'], refit='f1', return_train_score = False)
grid.fit(scaler.transform(X_train), Y_train)

In [ ]:
grid_mean_scores = grid.cv_results_['mean_test_score']
print(grid.cv_results_['rank_test_score'])

In [ ]:
print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)

In [ ]:
mejor_pred_train = grid.best_estimator_.predict(scaler.transform(X_train))
mejor_pred_test = grid.best_estimator_.predict(scaler.transform(X_test))

In [ ]:
cm_train = confusion_matrix(Y_train, mejor_pred_train, labels = knn_model.classes_)
cm_test = confusion_matrix(Y_test, mejor_pred_test, labels = knn_model.classes_)
cm_train_norm = confusion_matrix(Y_train, mejor_pred_train, labels = knn_model.classes_, normalize = 'true')
cm_test_norm = confusion_matrix(Y_test, mejor_pred_test, labels = knn_model.classes_, normalize = 'true')

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize = (13, 10))
ConfusionMatrixDisplay(confusion_matrix = cm_train, display_labels = knn_model.classes_).plot(ax = ax1, values_format = ',.0f')
ConfusionMatrixDisplay(confusion_matrix = cm_train_norm, display_labels = knn_model.classes_).plot(ax = ax2, values_format = '.3f')
ConfusionMatrixDisplay(confusion_matrix = cm_test, display_labels = knn_model.classes_).plot(ax = ax3, values_format = ',.0f')
ConfusionMatrixDisplay(confusion_matrix = cm_test_norm, display_labels = knn_model.classes_).plot(ax = ax4, values_format = '.3f')

ax1.set_title('Train')
ax2.set_title('Train (normalized)')
ax3.set_title('Test')
ax4.set_title('Test (normalized)')
plt.show()

In [ ]:
pd.DataFrame([
    {
        'Train': f1_score(Y_train, mejor_pred_train),
        'Test': f1_score(Y_test, mejor_pred_test)
    }
], index = ['F1'])

### Serialización del modelo

In [ ]:
filename = 'KNN/best_model_v1.0.pkl'

In [ ]:
pickle.dump(grid.best_estimator_, open(filename, 'wb'))